In [ ]:
import pickle
from typing import List

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp, odeint
from scipy.interpolate import interp1d

# helper functions for plotting
from utils import plot_throw2

# set jupyter configurations
%matplotlib inline
%config InlineBackend.figure_format='retina'
default_dpi = mpl.rcParamsDefault["figure.dpi"]
mpl.rcParams["figure.dpi"] = default_dpi * 1.2

## Simulator: Throwing a ball in air

A ballistic throw in air is modelled via a differential equation:

$\frac{d}{dt} x = v_x$

$a_x\frac{d}{dt} v_x = -\frac{k}{m} \cdot v_x \cdot s$

$\frac{d}{dt} z = v_z$

$a_z\frac{d}{dt} v_z = -\frac{k}{m} \cdot v_z \cdot s - g$

Here, $g$ is the gravitational acceleration, $k$ is the drag constant, $m$ is the mass of the object and $s=|v|= \sqrt{v_x^2 + v_z^2}$.


In [ ]:
def throw(
    speed: float,
    angle: int,
    drag: float,
    r: float = 0.050,
    m: float = 0.2,
    h_noise: float = 3.0,
    ρ: float = 1.225,
    g: float = 9.81,
) -> dict:
    """Simulate the throw of a ball.
    
    Following https://scipython.com/book2/chapter-8-scipy/examples/a-projectile-with-air-resistance/
    
    Args:
        speed: magnitude of initial speed (m/s).
        angle: launch angle with horizontal (degrees)
        drag: drag coefficient
        r: projectile radius (m)
        m: projectile mass (kg)
        h_noise: std of measurements of altitude (m)
        ρ: air density (default is at sea surface, 15C)
        g: gravitational acceleration (default is average at surface of Earth)
    
    Returns:
        simulation: simulation results containing distance travelled
            and height as a function of time.
    """

    k = 0.5 * drag * ρ * (np.pi * r**2)  # drag constant, proportional to area

    def deriv(t, u):
        """Return right-hand side of ODE system for the throw."""
        # see diagram at e.g. http://www.physics.smu.edu/fattarus/ballistic.html

        #### INSERT YOUR CODE HERE ####
        x, v_x, z, v_z = u
        speed = np.hypot(v_x, v_z)
        a_x, a_z = -k / m * speed * v_x, -k / m * speed * v_z - g

        ###############################

        return v_x, a_x, v_z, a_z

    # position and velocity components at launch
    x0, z0 = 0, 0
    rad_angle = np.radians(angle)
    v0_x, v0_z = speed * np.cos(rad_angle), speed * np.sin(rad_angle)

    # integration grid
    t = np.linspace(0, 400, 4000)

    # solve initial value problem (ivp) for distance traveled(t) and height(t)

    # df/dt = f(t,y); f(t_i) = y_i,

    #### INSERT YOUR CODE HERE ####

    solution = solve_ivp(
        deriv, t_span=(t[0], t[-1]), y0=(x0, v0_x, z0, v0_z), dense_output=True
    )  # dense => continuous solution

    ###############################

    # evaluate solution to obtain distance and height at each time point
    d, _, h, _ = solution.sol(t)

    # the simulator always uses the same time grid, interpolate to the same distance grid
    f = interp1d(d, h, bounds_error=False)

    d_target = np.linspace(0, 150, 100)

    h_target = f(d_target)

    # add noise to make the process stochastic
    x = h_target + h_noise * np.random.randn(d_target.shape[0])

    # to obtain the params from the interactive plot, we need to return parameters here as well
    return dict(θ=(speed, angle, drag, r, m, h_noise, ρ, g), d=d_target, x=x)

If you run the cell below with throwing speed 40 m/s, angle 30 degrees and drag coefficient 0.13, you obtain circles that directly overlay $x_o$ (i.e., the original simulation).

In [ ]:
with open("throw-x_o-1.pickle", "rb") as f:
    d_o, x_o = pickle.load(f)

sim = throw(40, 30, 0.13, h_noise=0)  # run the simulation without noise

d, x = sim["d"], sim["x"]

plot_throw2(d_o, x_o, d, x)

a) Familiarize yourself with the simulator by running it with different parameter settings for speed, angle and drag.

b) Find a different parameter set than the original parameter set (speed 40 m/s, angle 30 degrees and drag 0.13) that gives you the same final distance traveled. This online simulator can help you: http://www.physics.smu.edu/fattarus/ballistic.html

1. When models have different parameter sets that give the same solution, we say that the models show degeneracy (or that they have degenerate solutions).
2. This is a property of many models in natural sciences (e.g., Physics, Biology...), and is the subject of active research: e.g., how to analytically or numerically find these degenerate solutions, or what these degenerate solutions mean about the reality they are modelling.